# Описание задачи
Это практическая работа по использованию API различных сервисов с помощью библиотеки requests. Нужно получить некоторые данные и сохранить их.

In [1]:
!pip install requests

Реализуйте указанное в одном питон файле и прикрепите его:

Напишите функцию get_male_female_count(), которая  выполняет GET запрос к данному URL (https://rickandmortyapi.com/api/character?page=1) и возвращает количество женских и мужских персонажей в следующем формате {‘male’: 321, ‘female’: 123}.

Напишите функцию get_character_by_status(character_status), которая принимает на вход статус персонажа в строковом формате, выполняет GET запрос к данному методу (https://rickandmortyapi.com/api/character/) с параметром status, значение которого равно character_status, и возвращает список имен персонажей. 

Возможные статусы персонажей: alive, dead or unknown. 

Название персонажа возвращается под ключем name в объекте списка results: 

"results": [

    {

      "id": 1,

      "name": "Rick Sanchez",

      "status": "Alive",

      ….

      },

…



Подробнее о методе можно узнать по ссылке (https://rickandmortyapi.com/documentation/#filter-characters)

In [7]:
def get_male_female_count():
    import requests
    
    male_counter = 0
    female_counter = 0
    
    response = requests.get('https://rickandmortyapi.com/api/character?page=1t')
    for character in response.json()['results']:
        if character['gender'] == 'Male':
            male_counter += 1
        elif character['gender'] == 'Female':
            female_counter += 1

    counts = {'male': male_counter, 'female': female_counter}
    return counts

def get_character_by_status(character_status):
    import requests
    
    statuses = ['alive', 'dead', 'unknown']
    if character_status not in statuses:
        print("""Incorrect status. Possible statuses: 'alive', 'dead', 'unknown'""")
        return
    
    response = requests.get('https://rickandmortyapi.com/api/character/', params={"status" : character_status})
    characters_list = []
    for character in response.json()['results']:
        characters_list.append(character['name'])
    return characters_list

Напишите код, который соберет данные в JSON файл о 1000 фильмах жанра комедия КиноПоиска 2000 года релиза, отсортированных по уменьшению рейтинга кинопоиска: название, длительность, страна производитель. . Если всего фильмов меньше 1000, то нужно собрать все, иначе только 1000.

In [19]:
import requests
import json

TOKEN = '5CFBRQE-1J6MFG0-QEQWCAY-88GJ4M9'
URL = 'https://api.kinopoisk.dev/v1.4/movie'

session = requests.Session()
session.headers.update({'accept': 'application/json', 'X-API-KEY': TOKEN})

result_movies = []
response = session.get(URL,
    params={'type': 'movie',
             'genres.name': 'комедия',
             'year': 2000,
             'selectFields' : ['name', 'movieLength', 'countries'],
             'sortField': 'rating.kp',
             'sortType' : -1,
             'page' : 1,
             'limit':250})
if response.status_code:
    result_response = response.json()
    result_movies += result_response.get('docs', [])
    count_pages = result_response.get('pages',0)
    if count_pages > 4:
        count_pages = 5
    for page in range(2, count_pages):
        response = session.get(URL,
            params={'type': 'movie',
                     'genres.name': 'комедия',
                     'year': 2000,
                     'selectFields' : ['name', 'movieLength', 'countries'],
                     'sortField': 'rating.kp',
                     'sortType' : -1,
                     'page' : page,
                     'limit':250})
        if response.status_code:
            result_movies += result_response.get('items', [])

with open('comedies.json', 'w', encoding='utf-8') as f:
    json.dump(result_movies, f)

Авторизуйтесь в сервисе, получите API-ключ. Вам необходимо выполнить запрос на получение текущей температуры в вашем городе (API метод /current.json) и вывести в консоль: название города, температуру (в цельсиях), температура ощущается как (в цельсиях)

In [25]:
import requests
import json

TOKEN = 'b9da536380d54241add150855241102'
URL = 'http://api.weatherapi.com/v1/current.json'

session = requests.Session()
session.headers.update({'key': TOKEN})

response = session.get(URL,
    params={'q':'Penza'})

if response.status_code:
    result_response = response.json()

city = result_response['location']['name']
temp_c = result_response['current']['temp_c']
feelslike_c = result_response['current']['feelslike_c']

print(city, temp_c, feelslike_c)

Penza -13.8 -21.0


Напишите код, который соберет следующие данные о 1000 работодателей Алтайского края, которые имеют на текущий момент открытые вакансии: id, название работодателя, ссылки на вакансии, кол-во открытых вакансий. Данные должны быть отсортированы по кол-ву вакансий (от большего к меньшему) и собраны в csv файл. Для выполнения задания не нужно регистрировать приложение и использовать токен.

In [12]:
import requests
import pandas as pd

url_areas = 'https://api.hh.ru/areas'
url_emp = 'https://api.hh.ru/employers'

response = requests.get(url_areas)
if response.status_code:
    obj_ru = next(data for data in response.json() if data['name'] == 'Россия')
    region_id = next(area['id'] for area in obj_ru['areas'] if area['name'] == 'Алтайский край')

result = []
for page in range(0, 10):
    response_emp = requests.get(url_emp,
                            params = {
                                'page': page,
                                'per_page': 100,
                                'area': region_id,
                                'only_with_vacancies': True,
                                'sort_by': 'by_vacancies_open'})
    if response_emp.status_code:
        result += response_emp.json().get('items', [])

with open('Алтайский край.csv', 'w', encoding = 'utf-8') as file:
    names = ['id', 'name', 'vacancies_url', 'open_vacancies']
    dict_writer = csv.DictWriter(file, names)
    dict_writer.writeheader()
    for row in result:
        filtered_row = [{key: row[key] for key in row if key in names}]
        dict_writer.writerows(filtered_row)

In [13]:
df = pd.read_csv('Алтайский край.csv', encoding='utf-8')
df.head()

,id,name,vacancies_url,open_vacancies
0,57770,Мария-Ра,https://api.hh.ru/vacancies?employer_id=57770,403
1,91926,НОВЭКС,https://api.hh.ru/vacancies?employer_id=91926,180
2,620502,Крупнейший дистрибьютор продуктов питания в Ал...,https://api.hh.ru/vacancies?employer_id=620502,154
3,1916194,Сеть магазинов Дальнобойщик,https://api.hh.ru/vacancies?employer_id=1916194,152
4,69789,"АНИКС, Торговый Двор",https://api.hh.ru/vacancies?employer_id=69789,133


Напишите код, который соберет следующие данные в CSV файл о всех вакансиях за последнюю неделю по ключевому слову “Аналитик” с сайта https://superjob.ru: ссылка на вакансию, название вакансии, название работодателя, город, заработная плата, заработная плата от, заработная плата до, должностные обязанности, дата публикации, является ли вакансия архивной.

In [31]:
import requests
import csv
import time

URL = 'https://api.superjob.ru/2.0/vacancies/'
TOKEN = 'v3.r.138184723.f8dda1adff9f2999801b3024348f824ecaf743e2.1e14f366c6602f630f9407f17f13534edbfba503'

vacancies = []
page=0

while True:  
    response = requests.get(URL,                                       
        headers = {'X-Api-App-Id': TOKEN, 'accept':'application/json'},
        params={'keywords': [1, 'or','Аналитик'],  
                'date_published_from': int(time.mktime(time.strptime('2024-03-13T00:00:00.00+03:00', "%Y-%m-%dT%H:%M:%S.%f%z"))),
                'date_published_to': int(time.mktime(time.strptime('2024-03-18T00:00:00.00+03:00', "%Y-%m-%dT%H:%M:%S.%f%z"))),
                'page':page,
                'count':100})
    vacancies_page = response.json().get('objects', [])
    if vacancies_page == []:
        break
    vacancies += vacancies_page
    page += 1

headers = ['ссылка на вакансию',
           'название вакансии',
           'название работодателя',
           'город',
           'заработная плата',
           'заработная плата от',
           'заработная плата до',
           'должностные обязанности',
           'дата публикации',
           'является ли вакансия архивной']

with open('superjob_analysts.csv', mode='w', encoding='utf-8', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()
    for vacancy in vacancies:
        writer.writerow({
            'ссылка на вакансию': vacancy['link'],
            'название вакансии': vacancy['profession'],
            'название работодателя': vacancy['firm_name'],
            'город': vacancy['town']['title'],
            'заработная плата от': vacancy['payment_from'], 
            'заработная плата до': vacancy['payment_to'], 
            'должностные обязанности': vacancy['candidat'], 
            'дата публикации': vacancy['date_pub_to'], 
            'является ли вакансия архивной': vacancy['is_closed']
        })
        
print('Всего страниц', page+1)        
df = pd.read_csv('superjob_analysts.csv', encoding='utf-8')
df.head()

Всего страниц 3


,ссылка на вакансию,название вакансии,название работодателя,город,заработная плата,заработная плата от,заработная плата до,должностные обязанности,дата публикации,является ли вакансия архивной
0,https://spb.superjob.ru/vakansii/analitik-1s-4...,Аналитик 1С,Полиметалл,Санкт-Петербург,NaN,0,0,Обязанности: \n• Консультирование пользователе...,1711631705,False
1,https://vladivostok.superjob.ru/vakansii/vedus...,Ведущий финансовый аналитик,СКБ Приморья «Примсоцбанк»,Владивосток,NaN,60000,80000,Обязанности:\n• Ведение общебанковского плана ...,1713159312,False
2,https://murmansk.superjob.ru/vakansii/rukovodi...,Руководитель отдела аналитики и маркетинга,МТС,Мурманск,NaN,80000,0,МТС — это мультисервисная цифровая экосистема....,1713268833,False
3,https://balashiha.superjob.ru/vakansii/himik-a...,Химик-аналитик лаборатории ОКК,"АО ""АГРОБИОПРОМ""",Балашиха,NaN,90000,100000,Условия работы:\n\n• Заработная плата выплачив...,1711706600,False
4,https://tula.superjob.ru/vakansii/himik-analit...,"Химик-аналитик, лаборант",Интерфарма,Тула,NaN,24000,30000,Обязанности:\n• Отбор проб поступающего сырья ...,1711009204,False


Техническое задание от заказчика:

Заказчик попросил проанализировать, в каких сообществах социальной сети ВКонтакте ему будет выгодно разместить рекламу своего цветочного бизнеса.

Необходимо собрать следующие данные:
- ID сообщества,чтобы использовать в рекламе;
- Название сообщества, чтобы оценить подходит группа или нет;
- Описание сообщества, для более точного определения группы (не обязательно);
- Является ли сообщество закрытым или открытым;
- Количество подписчиков, чтобы оценить размер сообщества;
- Контакты сообществ, в которых вы хотите разместить рекламу;


Также были выделены следующие важные факторы:

- В качестве ключевых слов для поиска сообществ были выбраны: цветы, флористика, магазин цветов
- Сообщества должны быть отсортированы по по количеству пользователей (по убыванию);
- Поиск сообществ должен происходить только в пределах города Омска;
- Данные должны быть переданы в CSV файле

In [ ]:
!pip install vk_api

In [2]:
import vk_api
import csv
import pandas as pd

token = 'token'
session = vk_api.VkApi(token=token)
vk = session.get_api()

city_id = vk.database.getCities(
    country_id=1, q='Омск')['items']['title'=='Омск']['id']

groups = vk.groups.search(
    city_id=city_id, q='цветы, флористика, магазин цветов', sort=6)

group_ids = []
for group in groups['items']:
    group_ids.append(group['id'])

groups_info = vk.groups.getById(
    group_ids=group_ids, fields=['description', 'members_count', 'contacts'])

with open('vk_flowers_omsk.csv', 'w', encoding = 'utf-8') as file:
    names = ['id', 'name', 'description', 'is_closed', 'members_count', 'contacts']
    dict_writer = csv.DictWriter(file, names)
    dict_writer.writeheader()
    for row in groups_info:
        filtered_row = [{key: row[key] for key in row if key in names}]
        dict_writer.writerows(filtered_row)

df = pd.read_csv('vk_flowers_omsk.csv', encoding='utf-8')
df.head()

,id,name,description,is_closed,members_count,contacts
0,23992048,LAVANDA - цветы в Омске с доставкой,🌷 Доставка цветов в Омске\r\n\r\n🌻 Стильные бу...,0,61995,[]
1,45854603,Омск / zOrg shop,Вас приветствует Zorg !\r\nЯ рад предложить Ва...,0,41895,"[{'user_id': 555259698, 'desc': 'По всем вопро..."
2,138086573,Цветы в Омске | Доставка цветов - Клумба55,КРУГЛОСУТОЧНАЯ доставка цветов\r\nСобираем бук...,0,36350,"[{'user_id': 408424668, 'desc': 'Я принимаю за..."
3,63180778,"LUCKY KIDS ZONE (ZARA, H&M...)",🌈Дорогие покупатели!!! Рады приветствовать Вас...,0,30914,[]
4,67066486,Цветы в Омске. Кактус (Kaktus),Цветочный Магазин Кактус Омск. Все мы любим цв...,0,27992,"[{'desc': 'менеджер', 'phone': '79960751994', ..."
